In [2]:
from time_series_transform.time_series_transformer import Time_Series_Transformer
import pandas as pd
import numpy as np

In [6]:
df = pd.read_csv('../Data/Train/sales_train_validation.csv').sample(20)
dimList = ['item_id', 'dept_id', 'cat_id', 'store_id', 'state_id']

In [7]:
tst = Time_Series_Transformer(df,dimList)

In [8]:
gen = tst.np_to_time_tensor_generator(10)

In [10]:
pass

In [1]:
from time_series_transform.sequence_transfomer import *
import numpy as np
a = np.array([1,2,3,4,5,6,7,8,9,10,11,12])
Moving_Average(window_size=6).Call(a)

array([nan, nan, nan, nan, nan, 3.5, 4.5, 5.5, 6.5, 7.5, 8.5, 9.5])

In [20]:
b = a.reshape((-1,2,1))

In [37]:
c = np.dstack((b,b))
np.dstack((c,b)).shape

(6, 2, 3)

In [33]:
b.shape

(6, 2, 1)